In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, hour

Create Spark Session

In [2]:
spark = (
    SparkSession.builder.appName('MAST30034 Project 1')
    .config('spark.sql.repl.eagerEval.enabled', True) 
    .config('spark.sql.parquet.cacheMetadata', 'true')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "4g") 
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 19:22:35 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/19 19:22:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 19:22:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read in parquet files as spark dataframes - 3 months at a time (time frame my computer can handle)

In [3]:
# Define the years and months
years = list(range(2016, 2020))
months = list(range(1, 13, 3))

# Create a list to hold the DataFrames
yellow_taxi_dataframes = []

# Iterate through years and months to read Parquet files
for year in years:
    for start_month in months:
        end_month = start_month + 2 if start_month + 2 <= 12 else 12
        
        sdf_yellow = spark.read.parquet(f'../data/landing/yellow_taxi/{year}_{start_month:02}-{end_month:02}/*')
        yellow_taxi_dataframes.append(sdf_yellow)

Count the number of features intially

In [6]:
num_columns = len(yellow_taxi_dataframes[0].columns)
print("Number of columns:", num_columns)

Number of columns: 19


Function to count and display size of data

In [ ]:
def calculate_total_rows(dataframes):
    """
    Calculates the total number of rows in a list of Spark DataFrames
    Inputs:
        List of Spark DataFrames for which to calculate the total rows
    """
    total_rows = 0
    for df in dataframes:
        total_rows += df.count()
    print("Size of data:", total_rows)

Count size of imported taxi data

In [5]:
# Use function to count data and display it
calculate_total_rows(yellow_taxi_dataframes)

Size of initial import: 432101963


Remove inconsistent data

In [6]:
# Define the filter conditions
filter_conditions = (
    (F.col("Fare_amount") >= 2.5) &
    (F.col("Passenger_count") >= 1) &
    (F.col("PULocationID").between(1, 263)) &
    (F.col("DOLocationID").between(1, 263)) &
    (F.col("tpep_pickup_datetime") <= F.col("tpep_dropoff_datetime"))
)

# 3 months data
# Create a list to store the filtered DataFrames
filtered_yellow_taxi_dataframes = []

# Apply the filter to each DataFrame and store the result
for df in yellow_taxi_dataframes:
    filtered_df = df.filter(filter_conditions)
    filtered_yellow_taxi_dataframes.append(filtered_df)

Create Pick Up hour (time of day) column

In [7]:
# List to store modified DataFrames with the "weekend" column
full_taxi_periods = []

# Loop through the list of DataFrames and add the "pickup_hour" column
for taxi_df in filtered_yellow_taxi_dataframes:
    full_modified_taxi_df = taxi_df.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))
    full_taxi_periods.append(full_modified_taxi_df)

Count size of first filtering of taxi data

In [ ]:
# Use function to count size of data after filtering
size_of_data = calculate_total_rows(filtered_yellow_taxi_dataframes)

Size of data after removing inconsistent data: 420964216


Removing unneccesary columns from yellow taxi data. In other notebooks will look to aggregate data into hours. Also in the models, I want to predict rates of pickups per hour. Logically I can't use any data made from after pick up.

In [10]:
# Select specific columns and display the result
selected_columns = ['VendorID', 'tpep_pickup_datetime', 'PULocationID', 'DOLocationID',
                    'pickup_hour']

# Removing columns that aren't being used from all yellow taxi data
# Create a list to store the new selected DataFrames
selected_yellow_taxi_dataframes = []

# Iterate through each DataFrame and apply column selection
for df in full_taxi_periods:
    selected_df = df.select(*selected_columns)
    selected_yellow_taxi_dataframes.append(selected_df)

Saving data segemented into 3 months

In [12]:
# Define a list of output paths for each Parquet file
output_paths = [
    "../data/raw/yellow_taxi_reduced/2016_01_to_03.parquet",
    "../data/raw/yellow_taxi_reduced/2016_04_to_06.parquet",
    "../data/raw/yellow_taxi_reduced/2016_07_to_09.parquet",
    "../data/raw/yellow_taxi_reduced/2016_10_to_12.parquet",
    "../data/raw/yellow_taxi_reduced/2017_01_to_03.parquet",
    "../data/raw/yellow_taxi_reduced/2017_04_to_06.parquet",
    "../data/raw/yellow_taxi_reduced/2017_07_to_09.parquet",
    "../data/raw/yellow_taxi_reduced/2017_10_to_12.parquet",
    "../data/raw/yellow_taxi_reduced/2018_01_to_03.parquet",
    "../data/raw/yellow_taxi_reduced/2018_04_to_06.parquet",
    "../data/raw/yellow_taxi_reduced/2018_07_to_09.parquet",
    "../data/raw/yellow_taxi_reduced/2018_10_to_12.parquet",
    "../data/raw/yellow_taxi_reduced/2019_01_to_03.parquet",
    "../data/raw/yellow_taxi_reduced/2019_04_to_06.parquet",
    "../data/raw/yellow_taxi_reduced/2019_07_to_09.parquet",
    "../data/raw/yellow_taxi_reduced/2019_10_to_12.parquet",
]

# Iterate through each DataFrame and its corresponding output path
for i in range(len(output_paths)):
    selected_yellow_taxi_dataframes[i].write.parquet(output_paths[i])
    
# Stop the SparkSession
spark.stop()